In [9]:
import numpy as np
import sys
import os

from dplearning.book_material.common.layers import Convolution, Relu, Pooling
from dplearning.book_material.common.util import im2col
from dplearning.book_material.common.layers import SoftmaxWithLoss

sys.path.append('./book_material')
from common.util import *

In [33]:
from dplearning.book_material.common.layers import Affine


class SimpleConvNet:
    def __init__(self,input_dim=(1,28,28),conv_param={'filter_num':30,'filter_size':5,'pad':0,'stride':1},hidden_size=100,output_size=10,weight_init_std=0.01):
        self.input_dim = input_dim
        self.hidden_size = hidden_size
        filter_num = conv_param['filter_num']
        filter_size = conv_param['filter_size']
        filter_pad = conv_param['pad']
        filter_stride = conv_param['stride']
        input_size = input_dim[1]
        conv_output_size = (input_size - filter_size + 2*filter_pad) / filter_stride + 1
        ##使用 2x2 池化窗口，步长为 2（无填充） 必须要展开成1*n的张量
        pool_output_size = int(filter_num * (conv_output_size/2) *(conv_output_size/2))
        self.params={}
        self.params['W1']=weight_init_std * np.random.randn(filter_num,input_dim[0],filter_size,filter_size)
        self.params['b1']=np.zeros(filter_num)
        self.params['W2']=weight_init_std * np.random.randn(pool_output_size,hidden_size)
        self.params['b2']=np.zeros(hidden_size)
        self.params['W3']=weight_init_std * np.random.randn(hidden_size,output_size)
        self.params['b3']=np.zeros(output_size)
        self.layers={}
        self.layers['Conv1']=Convolution(self.params['W1'],self.params['b1'],conv_param['stride'],conv_param['pad'])
        self.layers['Relu1']=Relu()
        self.layers['Pool']=Pooling(2,2,2)

        self.layers['Affine1']=Affine(self.params['W2'],self.params['b2'])
        self.layers['Relu2']=Relu()
        self.layers['Affine2']=Affine(self.params['W3'],self.params['b3'])
        self.last_layer=SoftmaxWithLoss()
    def predict(self,x):
        for i in self.layers.keys():
            x=self.layers[i].forward(x)
        return x
    def loss(self,x):
        y_pred=self.predict(x)
        return self.last_layer.loss(y,y_pred)
    def gradient(self,x):
        self.loss(x)
        dout=1
        dout=self.last_layer.backward(dout)
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)
            #
        grads = {}
        grads['W1'] = self.layers['Conv1'].dW
        grads['b1'] = self.layers['Conv1'].db
        grads['W2'] = self.layers['Affine1'].dW
        grads['b2'] = self.layers['Affine1'].db
        grads['W3'] = self.layers['Affine2'].dW
        grads['b3'] = self.layers['Affine2'].db
        return grads

In [17]:
sys.path.append('./book_material')
from dataset.mnist import *
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label = True)

In [18]:
conv=SimpleConvNet()

In [25]:
result=np.argmax(conv.predict(x_test.reshape(10000,1,28,28)),axis=1)

In [28]:
result

array([7, 6, 6, ..., 1, 6, 1])

In [34]:
np.argmax(t_test,axis=1)

array([7, 2, 1, ..., 4, 5, 6])